In [1]:
%load_ext kamu

In [14]:
%%local
import pandas as pd
#pd.set_option('display.max_rows', 500)
#pd.set_option('display.max_columns', 500)
#pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', 1000)

In [1]:
%%local
import rattletrap as rt

rt.

In [2]:
%import_dataset replays

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,None,pyspark,idle,,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
%%sql
select * from replays

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [28]:
%%sql
with

player_stats_json as (
    select
        event_time,
        get_json_object(
            replay_json, 
            '$.header.body.properties.value.PlayerStats.value.array'
        ) as ps_json 
    from replays
),

player_stats_raw as (
    select
        event_time,
        explode(from_json(ps_json, '
            array<struct<value:struct<
                Name:struct<value:struct<str:string>>,
                Team:struct<value:struct<int:integer>>,
                bBot:struct<value:struct<bool:integer>>,
                Score:struct<value:struct<int:integer>>,
                Goals:struct<value:struct<int:integer>>,
                Assists:struct<value:struct<int:integer>>,
                Shots:struct<value:struct<int:integer>>,
                Saves:struct<value:struct<int:integer>>
            >>>
        ')) as stats
    from player_stats_json
),
                
player_stats as (
    select
        event_time,
        stats.value.name.value.str as name,
        stats.value.team.value.int as team,
        stats.value.bBot.value.bool as is_bot,
        stats.value.score.value.int as score,
        stats.value.goals.value.int as goals,
        stats.value.assists.value.int as assists,
        stats.value.shots.value.int as shots,
        stats.value.saves.value.int as saves
    from player_stats_raw
)

select * from player_stats

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [64]:
%%sql
with

header_json as (
    select
        event_time,
        get_json_object(
            replay_json,
            '$.header.body.properties.value'
        ) as header_json
    from replays
),

header_raw as (
    select
        event_time,
        from_json(header_json, '
            struct<
                Id:struct<value:struct<str:string>>,
                Date:struct<value:struct<str:string>>,
                MapName:struct<value:struct<name:string>>,
                TeamSize:struct<value:struct<int:integer>>,
                UnfairTeamSize:struct<value:struct<int:integer>>
            >
        ') as header
    from header_json
)
                
select
    cast(unix_timestamp(header.Date.value.str, "yyyy-MM-dd HH-mm-ss") as timestamp) as event_time,
    header.Id.value.str as match_id,
    header.MapName.value.name as map_name,
    header.TeamSize.value.int as team_size,
    header.UnfairTeamSize.value.int as unfair_team_size
from header_raw

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [61]:
%%sql
--select cast(unix_timestamp("2022-10-06 20-15-18", "yyyy-M-d h-m-s") as timestamp) as event_time
select cast(unix_timestamp("2022-10-06 20-15-18", "yyyy-MM-dd HH-mm-ss") as timestamp) as event_time

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()